In [106]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [107]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [108]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [109]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///../Resources/hawaii.sqlite")

In [110]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [111]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [112]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [113]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [114]:
# Find the most recent date in the data set.
most_recent_date = session.query(measurement.date).order_by(measurement.date.desc()).first()
print(most_recent_date)

('2017-08-23',)


In [115]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
one_year_ago = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print(one_year_ago)
one_year_ago


2016-08-23


datetime.date(2016, 8, 23)

In [116]:

# Perform a query to retrieve the data and precipitation scores
results = session.query(measurement).filter(
    measurement.date >= one_year_ago,
    measurement.date <= dt.date(2017, 8, 23),
).all()

# Collect the data in lists
ids = []
stations = []
dates = []
precipitations = []
temperatures = []

for result in results:
    ids.append(result.id)
    stations.append(result.station)
    dates.append(result.date)
    precipitations.append(result.prcp)
    temperatures.append(result.tobs)


In [117]:
# Save the query results as a Pandas DataFrame
last_year = pd.DataFrame({
    'ID': ids,
    'Station': stations,
    'Date': dates,
    'Precipitation': precipitations,
    'Temperature': temperatures,
})

# Print or use the DataFrame as needed
print(last_year)

         ID      Station        Date  Precipitation  Temperature
0      2364  USC00519397  2016-08-23           0.00         81.0
1      2365  USC00519397  2016-08-24           0.08         79.0
2      2366  USC00519397  2016-08-25           0.08         80.0
3      2367  USC00519397  2016-08-26           0.00         79.0
4      2368  USC00519397  2016-08-27           0.00         77.0
...     ...          ...         ...            ...          ...
2225  19546  USC00516128  2017-08-19           0.09         71.0
2226  19547  USC00516128  2017-08-20            NaN         78.0
2227  19548  USC00516128  2017-08-21           0.56         76.0
2228  19549  USC00516128  2017-08-22           0.50         76.0
2229  19550  USC00516128  2017-08-23           0.45         76.0

[2230 rows x 5 columns]


In [118]:

# Sort the dataframe by date
sorted_last_year = last_year.sort_values('date', ascending=False)

# Use Pandas Plotting with Matplotlib to plot the data


KeyError: 'date'

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close Session

In [ ]:
# Close Session
session.close()